In [5]:
import os
import time
import zipfile
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.random_projection import SparseRandomProjection

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Decision Tree with Manual Cross-Validation

def flastThreshold(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, params):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define Decision Tree model with given parameters
    dt_model = DecisionTreeClassifier(
        criterion=params.get('criterion', 'entropy'),
        max_depth=params.get('max_depth', None),
        min_samples_split=params.get('min_samples_split', 2),
        min_samples_leaf=params.get('min_samples_leaf', 1),
        max_features=params.get('max_features', None),
        random_state=42
    )

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize metrics storage for each threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_threshold = {threshold: {'f1': 0, 'accuracy': 0, 'precision': 0, 'recall': 0, 'mcc': 0}
                             for threshold in thresholds}
    successFold = 0

    for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
        X_train, X_test = Z[train_index], Z[test_index]
        y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

        if sum(y_train) == 0 or sum(y_test) == 0:
            print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
            continue

        # Train the model
        dt_model.fit(X_train, y_train)

        # Predict probabilities on test set
        y_pred_proba = dt_model.predict_proba(X_test)

        # Calculate metrics for each threshold
        for threshold in thresholds:
            y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

            # Calculate metrics for this threshold
            f1 = f1_score(y_test, y_pred, average='binary', zero_division=1)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='binary', zero_division=1)
            recall = recall_score(y_test, y_pred, average='binary', zero_division=1)
            mcc = matthews_corrcoef(y_test, y_pred)

            # Accumulate metrics for the current threshold
            metrics_per_threshold[threshold]['f1'] += f1
            metrics_per_threshold[threshold]['accuracy'] += accuracy
            metrics_per_threshold[threshold]['precision'] += precision
            metrics_per_threshold[threshold]['recall'] += recall
            metrics_per_threshold[threshold]['mcc'] += mcc

        successFold += 1

    if successFold == 0:
        print("No valid folds. Exiting.")
        return params, None

    # Average metrics over all folds
    for threshold in thresholds:
        metrics_per_threshold[threshold]['f1'] /= successFold
        metrics_per_threshold[threshold]['accuracy'] /= successFold
        metrics_per_threshold[threshold]['precision'] /= successFold
        metrics_per_threshold[threshold]['recall'] /= successFold
        metrics_per_threshold[threshold]['mcc'] /= successFold

    # Save the results for each threshold
    outFile = f"{combination_label}-params-dt-{n_splits}-folds-Threshold.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        # Write the header
        fo.write("threshold,accuracy,precision,recall,f1,mcc\n")
        # Write the data for each threshold
        for threshold in thresholds:
            fo.write(f"{threshold},{metrics_per_threshold[threshold]['accuracy']},{metrics_per_threshold[threshold]['precision']},"
                     f"{metrics_per_threshold[threshold]['recall']},{metrics_per_threshold[threshold]['f1']},{metrics_per_threshold[threshold]['mcc']}\n")

    print(f"Decision Tree analysis completed for {successFold} folds. Results saved to: {outFile}")
    return params, metrics_per_threshold

if __name__ == "__main__":

    params = {
        "criterion": "entropy",
        "max_depth": 300,
        "min_samples_split": 5,
        "min_samples_leaf": 1,
        "max_features": 'log2'
    }

    # Parameters setup for the first combination
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

    # Create separate result directories for equal and larger non-flaky combinations
    outDirEqual = "results/equal_flaky_nonflaky/"
    outDirLarger = "results/larger_nonflaky/"
    os.makedirs(outDirEqual, exist_ok=True)
    os.makedirs(outDirLarger, exist_ok=True)

    # Create separate extract directories for each combination to avoid file confusion
    extractDirEqual = "extracted/equal_flaky_nonflaky/"
    extractDirLarger = "extracted/larger_nonflaky/"
    os.makedirs(extractDirEqual, exist_ok=True)
    os.makedirs(extractDirLarger, exist_ok=True)

    # Perform Decision Tree analysis for the first combination (flaky vs smaller non-flaky)
    print("Starting Decision Tree analysis for flaky vs smaller non-flaky files (47 each)...")
    best_params_5folds_1, best_score_5folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 5, dim=100, eps=0.3, combination_label="equal", params=params)
    best_params_3folds_1, best_score_3folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 3, dim=100, eps=0.3, combination_label="equal", params=params)

    print("Best results for 5-fold on equal combination:")
    print(f"Best Parameters: {best_params_5folds_1}")
    print(f"Best F1 Score: {best_score_5folds_1}")

    print("Best results for 3-fold on equal combination:")
    print(f"Best Parameters: {best_params_3folds_1}")
    print(f"Best F1 Score: {best_score_3folds_1}")

    # Perform Decision Tree analysis for the second combination (flaky vs larger non-flaky)
    print("Starting Decision Tree analysis for flaky vs larger non-flaky files...")
    best_params_5folds_2, best_score_5folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 5, dim=100, eps=0.3, combination_label="larger", params=params)
    best_params_3folds_2, best_score_3folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 3, dim=100, eps=0.3, combination_label="larger", params=params)

    print("Best results for 5-fold on larger non-flaky combination:")
    print(f"Best Parameters: {best_params_5folds_2}")
    print(f"Best F1 Score: {best_score_5folds_2}")

    print("Best results for 3-fold on larger non-flaky combination:")
    print(f"Best Parameters: {best_params_3folds_2}")
    print(f"Best F1 Score: {best_score_3folds_2}")




Starting Decision Tree analysis for flaky vs smaller non-flaky files (47 each)...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Decision Tree analysis completed for 5 folds. Results saved to: equal-params-dt-5-folds-Threshold.csv
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Decision Tree analysis completed for 3 folds. Results saved to: equal-params-dt-3-folds-Threshold.csv
Best results for 5-fold on equal combination:
Best Parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best F1 Score: {0.1: {'f1': 0.6540756302521008, 'accuracy': 0.6596491228070175, 'precision': 0.6605844155844156, 'recall': 0.6555555555555556, 'mcc': 0.3219734115997638}, 0.2: {'f1': 0.6540756302521008, 'accuracy': 0.6596491228070175, 'precision': 0.6605844155844156, 'recall': 0.6555555555555556, 'mcc': 0.3219734115997638}, 0.30000000000000004: {'

Decision Tree


In [8]:
import os
import time
import zipfile
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.random_projection import SparseRandomProjection

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Decision Tree with Manual Cross-Validation

def flastThreshold(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, params):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define Decision Tree model with given parameters
    dt_model = DecisionTreeClassifier(
        criterion=params.get('criterion', 'entropy'),
        max_depth=params.get('max_depth', None),
        min_samples_split=params.get('min_samples_split', 2),
        min_samples_leaf=params.get('min_samples_leaf', 1),
        max_features=params.get('max_features', None),
        random_state=42
    )

       # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_fold = {fold+1: {threshold: {'f1': 0, 'accuracy': 0, 'precision': 0, 'recall': 0, 'mcc': 0}
                                 for threshold in thresholds}
                        for fold in range(n_splits)}
    successFold = 0

    for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
        X_train, X_test = Z[train_index], Z[test_index]
        y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

        if sum(y_train) == 0 or sum(y_test) == 0:
            print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
            continue

        # Train the model
        dt_model.fit(X_train, y_train)

        # Predict probabilities on test set
        y_pred_proba = dt_model.predict_proba(X_test)

        # Calculate metrics for each threshold for this fold
        for threshold in thresholds:
            y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

            # Calculate metrics for this threshold
            f1 = f1_score(y_test, y_pred, zero_division=1)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, , zero_division=1)
            recall = recall_score(y_test, y_pred, zero_division=1)
            mcc = matthews_corrcoef(y_test, y_pred)

            # Store metrics for the current threshold for this fold
            metrics_per_fold[fold+1][threshold]['f1'] = f1
            metrics_per_fold[fold+1][threshold]['accuracy'] = accuracy
            metrics_per_fold[fold+1][threshold]['precision'] = precision
            metrics_per_fold[fold+1][threshold]['recall'] = recall
            metrics_per_fold[fold+1][threshold]['mcc'] = mcc

        successFold += 1

    if successFold == 0:
        print("No valid folds. Exiting.")
        return params, None

    # Save the results for each threshold and fold
    outFile = f"{combination_label}-params-dt-{n_splits}-folds-Threshold-allKfold.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        # Write the header
        fo.write("fold,threshold,accuracy,precision,recall,f1,mcc\n")
        # Write the data for each fold and each threshold
        for fold in range(1, successFold + 1):
            for threshold in thresholds:
                fo.write(f"{fold},{threshold},{metrics_per_fold[fold][threshold]['accuracy']},{metrics_per_fold[fold][threshold]['precision']},"
                         f"{metrics_per_fold[fold][threshold]['recall']},{metrics_per_fold[fold][threshold]['f1']},{metrics_per_fold[fold][threshold]['mcc']}\n")

    print(f"Decision Tree analysis completed for {successFold} folds. Results saved to: {outFile}")
    return params, metrics_per_fold



if __name__ == "__main__":

    params = {
        "criterion": "entropy",
        "max_depth": 300,
        "min_samples_split": 5,
        "min_samples_leaf": 1,
        "max_features" : 'log2'
    }

    # Parameters setup for the first combination
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

    # Create separate result directories for equal and larger non-flaky combinations
    outDirEqual = "results/equal_flaky_nonflaky/"
    outDirLarger = "results/larger_nonflaky/"
    os.makedirs(outDirEqual, exist_ok=True)
    os.makedirs(outDirLarger, exist_ok=True)

    # Create separate extract directories for each combination to avoid file confusion
    extractDirEqual = "extracted/equal_flaky_nonflaky/"
    extractDirLarger = "extracted/larger_nonflaky/"
    os.makedirs(extractDirEqual, exist_ok=True)
    os.makedirs(extractDirLarger, exist_ok=True)

    # Perform Decision Tree analysis for the first combination (flaky vs smaller non-flaky)
    print("Starting Decision Tree analysis for flaky vs smaller non-flaky files (47 each)...")
    best_params_5folds_1, best_score_5folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 5, dim=100, eps=0.3, combination_label="equal", params=params)
    best_params_3folds_1, best_score_3folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 3, dim=100, eps=0.3, combination_label="equal", params=params)

    print("Best results for 5-fold on equal combination:")
    print(f"Best Parameters: {best_params_5folds_1}")
    print(f"Best F1 Score: {best_score_5folds_1}")

    print("Best results for 3-fold on equal combination:")
    print(f"Best Parameters: {best_params_3folds_1}")
    print(f"Best F1 Score: {best_score_3folds_1}")

    # Perform Decision Tree analysis for the second combination (flaky vs larger non-flaky)
    print("Starting Decision Tree analysis for flaky vs larger non-flaky files...")
    best_params_5folds_2, best_score_5folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 5, dim=100, eps=0.3, combination_label="larger", params=params)
    best_params_3folds_2, best_score_3folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 3, dim=100, eps=0.3, combination_label="larger", params=params)

    print("Best results for 5-fold on larger non-flaky combination:")
    print(f"Best Parameters: {best_params_5folds_2}")
    print(f"Best F1 Score: {best_score_5folds_2}")

    print("Best results for 3-fold on larger non-flaky combination:")
    print(f"Best Parameters: {best_params_3folds_2}")
    print(f"Best F1 Score: {best_score_3folds_2}")



SyntaxError: invalid syntax (1414966639.py, line 131)

Decition Tree - all Hyperparameter

In [13]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.random_projection import SparseRandomProjection
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Manual Grid Search Decision Tree with Cross-Validation

def flastThreshold(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
        # Initialize DecisionTreeClassifier with current hyperparameter combination
        dt_model = DecisionTreeClassifier(
            criterion=param_dict['criterion'],
            max_depth=param_dict['max_depth'],
            min_samples_split=param_dict['min_samples_split'],
            min_samples_leaf=param_dict['min_samples_leaf'],
            max_features=param_dict['max_features'],
            random_state=42
        )
        
        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the model
            dt_model.fit(X_train, y_train)

            # Predict probabilities on test set
            y_pred_proba = dt_model.predict_proba(X_test)

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred, zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred,  zero_division=1)
                recall = recall_score(y_test, y_pred,  zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'criterion': param_dict['criterion'],
                    'max_depth': param_dict['max_depth'],
                    'min_samples_split': param_dict['min_samples_split'],
                    'min_samples_leaf': param_dict['min_samples_leaf'],
                    'max_features': param_dict['max_features'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return param_dict, None

    # Save the results for each combination, threshold, and fold
    df_results = pd.DataFrame(metrics_per_combination)
    outFile = f"{combination_label}-params-dt-{n_splits}-folds-Threshold-allKfold-wogridsearch.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Decision Tree analysis completed. Results saved to: {outFile}")
    return param_dict, df_results



if __name__ == "__main__":

    param_grid = {
        'criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
        'max_depth': [None, 10, 30, 50, 100, 300, 500],  # Maximum depth of each tree
        'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2, 5, 10],  # Minimum number of samples required to be at a leaf node
        'max_features': [None, 'sqrt', 'log2'],  # Number of features to consider when looking for the best split
    }

    # Parameters setup for the first combination
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

    # Create separate result directories for equal and larger non-flaky combinations
    outDirEqual = "results/equal_flaky_nonflaky/"
    outDirLarger = "results/larger_nonflaky/"
    os.makedirs(outDirEqual, exist_ok=True)
    os.makedirs(outDirLarger, exist_ok=True)

    # Create separate extract directories for each combination to avoid file confusion
    extractDirEqual = "extracted/equal_flaky_nonflaky/"
    extractDirLarger = "extracted/larger_nonflaky/"
    os.makedirs(extractDirEqual, exist_ok=True)
    os.makedirs(extractDirLarger, exist_ok=True)

    # Perform Decision Tree analysis for the first combination (flaky vs smaller non-flaky)
    print("Starting Decision Tree analysis for flaky vs smaller non-flaky files (47 each)...")
    best_params_5folds_1, df_results_5folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 5, dim=100, eps=0.3, combination_label="equal", param_grid=param_grid)
    
    print("Best results for 5-fold on equal combination:")
    print(df_results_5folds_1)

    # Perform Decision Tree analysis for the second combination (flaky vs larger non-flaky)
    print("Starting Decision Tree analysis for flaky vs larger non-flaky files...")
    best_params_5folds_2, df_results_5folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 5, dim=100, eps=0.3, combination_label="larger", param_grid=param_grid)
    
    print("Best results for 5-fold on larger combination:")
    print(df_results_5folds_2)




Starting Decision Tree analysis for flaky vs smaller non-flaky files (47 each)...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2'}


Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'gini',

Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max

Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini

Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'gini', 'max_depth': 500, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'log2'}
Training with parameters: {'criterion'

...

Training with parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with par

Training with parameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters: {'cri

Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters

Training with parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2'}
Training with parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': None}
Training with parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
Training with parameters

Decision Tree analysis completed. Results saved to: larger-params-dt-5-folds-Threshold-allKfold-wogridsearch.csv
Best results for 5-fold on larger combination:
      criterion  max_depth  min_samples_split  min_samples_leaf max_features  \
0          gini        NaN                  2                 1         None   
1          gini        NaN                  2                 1         None   
2          gini        NaN                  2                 1         None   
3          gini        NaN                  2                 1         None   
4          gini        NaN                  2                 1         None   
...         ...        ...                ...               ...          ...   
22675   entropy      500.0                 10                10         log2   
22676   entropy      500.0                 10                10         log2   
22677   entropy      500.0                 10                10         log2   
22678   entropy      500.0              

Random Forest

In [9]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.random_projection import SparseRandomProjection
from sklearn.ensemble import RandomForestClassifier
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Manual Grid Search Random Forest with Cross-Validation

def flastThreshold(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
        # Initialize RandomForestClassifier with current hyperparameter combination
        rf_model = RandomForestClassifier(
            n_estimators=param_dict['n_estimators'],
            criterion=param_dict['criterion'],
            max_depth=param_dict['max_depth'],
            min_samples_split=param_dict['min_samples_split'],
            min_samples_leaf=param_dict['min_samples_leaf'],
            random_state=42,
            n_jobs=-1
        )
        
        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the model
            rf_model.fit(X_train, y_train)

            # Predict probabilities on test set
            y_pred_proba = rf_model.predict_proba(X_test)

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred,  zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred,  zero_division=1)
                recall = recall_score(y_test, y_pred, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'n_estimators': param_dict['n_estimators'],
                    'criterion': param_dict['criterion'],
                    'max_depth': param_dict['max_depth'],
                    'min_samples_split': param_dict['min_samples_split'],
                    'min_samples_leaf': param_dict['min_samples_leaf'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return param_dict, None

    # Save the results for each combination, threshold, and fold
    df_results = pd.DataFrame(metrics_per_combination)
    outFile = f"{combination_label}-params-rf-{n_splits}-folds-Threshold-allKfold-wogridsearch.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Random Forest analysis completed. Results saved to: {outFile}")
    return param_dict, df_results



if __name__ == "__main__":

    param_grid = {
        'n_estimators': [10, 50, 100, 300, 500],  # Number of trees
        'max_depth': [10, 30, 50, 100, 300, 500],  # Maximum depth of each tree
        'min_samples_split': [2, 5],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2],  # Minimum number of samples required to be at a leaf node
        'criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
    }

    # Parameters setup for the first combination
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

    # Create separate result directories for equal and larger non-flaky combinations
    outDirEqual = "results/equal_flaky_nonflaky/"
    outDirLarger = "results/larger_nonflaky/"
    os.makedirs(outDirEqual, exist_ok=True)
    os.makedirs(outDirLarger, exist_ok=True)

    # Create separate extract directories for each combination to avoid file confusion
    extractDirEqual = "extracted/equal_flaky_nonflaky/"
    extractDirLarger = "extracted/larger_nonflaky/"
    os.makedirs(extractDirEqual, exist_ok=True)
    os.makedirs(extractDirLarger, exist_ok=True)

    # Perform Random Forest analysis for the first combination (flaky vs smaller non-flaky)
    print("Starting Random Forest analysis for flaky vs smaller non-flaky files (47 each)...")
    best_params_5folds_1, df_results_5folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 5, dim=100, eps=0.3, combination_label="equal", param_grid=param_grid)
    
    print("Best results for 5-fold on equal combination:")
    print(df_results_5folds_1)

    # Perform Random Forest analysis for the second combination (flaky vs larger non-flaky)
    print("Starting Random Forest analysis for flaky vs larger non-flaky files...")
    best_params_5folds_2, df_results_5folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 5, dim=100, eps=0.3, combination_label="larger", param_grid=param_grid)
    
    print("Best results for 5-fold on larger combination:")
    print(df_results_5folds_2)


Starting Random Forest analysis for flaky vs smaller non-flaky files (47 each)...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameter

Training with parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'mi

Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 100, 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_

Training with parameters: {'n_estimators': 300, 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'm

Number of flaky documents: 47
Number of non-flaky documents: 254
Total number of documents: 301
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 10, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_le

Training with parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 50, 'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 50, 'max_depth': 50, 

Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 100, 'max_depth': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 100, 'max_

Training with parameters: {'n_estimators': 300, 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'gini'}
Training with parameters: {'n_estimators': 300, 'max_depth': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Training with parameters: {'n_estimators': 300, 'm

XGB

In [14]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.random_projection import SparseRandomProjection
from xgboost import XGBClassifier
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Manual Grid Search XGBoost with Cross-Validation

def flastThreshold(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
        # Initialize XGBClassifier with current hyperparameter combination
        xgb_model = XGBClassifier(
            eta=param_dict['eta'],
            max_depth=param_dict['max_depth'],
            n_estimators=param_dict['n_estimators'],
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=42,
            n_jobs=-1
        )
        
        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the model
            xgb_model.fit(X_train, y_train, verbose=False)

            # Predict probabilities on test set
            y_pred_proba = xgb_model.predict_proba(X_test)

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred,  zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred,  zero_division=1)
                recall = recall_score(y_test, y_pred,  zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'eta': param_dict['eta'],
                    'max_depth': param_dict['max_depth'],
                    'n_estimators': param_dict['n_estimators'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return param_dict, None

    # Save the results for each combination, threshold, and fold
    df_results = pd.DataFrame(metrics_per_combination)
    outFile = f"{combination_label}-params-xgb-{n_splits}-folds-Threshold-allKfold-wogridsearch.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"XGBoost analysis completed. Results saved to: {outFile}")
    return param_dict, df_results



if __name__ == "__main__":

    param_grid = {
        'eta': [0.01, 0.1, 0.3, 0.5],  # Learning rate
        'max_depth': [3, 5, 7, 10],    # Tree depth
        'n_estimators': [50, 100, 200, 300],  # Number of boosting rounds
    }

    # Parameters setup for the first combination
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

    # Create separate result directories for equal and larger non-flaky combinations
    outDirEqual = "results/equal_flaky_nonflaky/"
    outDirLarger = "results/larger_nonflaky/"
    os.makedirs(outDirEqual, exist_ok=True)
    os.makedirs(outDirLarger, exist_ok=True)

    # Create separate extract directories for each combination to avoid file confusion
    extractDirEqual = "extracted/equal_flaky_nonflaky/"
    extractDirLarger = "extracted/larger_nonflaky/"
    os.makedirs(extractDirEqual, exist_ok=True)
    os.makedirs(extractDirLarger, exist_ok=True)

    # Perform XGBoost analysis for the first combination (flaky vs smaller non-flaky)
    print("Starting XGBoost analysis for flaky vs smaller non-flaky files (47 each)...")
    best_params_5folds_1, df_results_5folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 5, dim=100, eps=0.3, combination_label="equal", param_grid=param_grid)
    
    print("Best results for 5-fold on equal combination:")
    print(df_results_5folds_1)

    # Perform XGBoost analysis for the second combination (flaky vs larger non-flaky)
    print("Starting XGBoost analysis for flaky vs larger non-flaky files...")
    best_params_5folds_2, df_results_5folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 5, dim=100, eps=0.3, combination_label="larger", param_grid=param_grid)
    
    print("Best results for 5-fold on larger combination:")
    print(df_results_5folds_2)



Starting XGBoost analysis for flaky vs smaller non-flaky files (47 each)...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Training with parameters: {'eta': 0.01, 'max_depth': 3, 'n_estimators': 50}


C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:41:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:41:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:41:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\

Training with parameters: {'eta': 0.01, 'max_depth': 3, 'n_estimators': 100}


C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:41:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:41:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:41:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\

Training with parameters: {'eta': 0.01, 'max_depth': 3, 'n_estimators': 200}


...

C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:42:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:42:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:42:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\

Training with parameters: {'eta': 0.5, 'max_depth': 10, 'n_estimators': 300}


C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:42:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:42:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:42:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost analysis completed. Results saved to: larger-params-xgb-5-folds-Threshold-allKfold-wogridsearch.csv
Best results for 5-fold on larger combination:
       eta  max_depth  n_estimators  fold  threshold  accuracy  precision  \
0     0.01          3            50     1        0.1  0.163934   0.163934   
1     0.01          3            50     1        0.2  0.770492   0.333333   
2     0.01          3            50     1        0.3  0.852459   1.000000   
3     0.01          3            50     1        0.4  0.836066   1.000000   
4     0.01          3            50     1        0.5  0.836066   1.000000   
...    ...        ...           ...   ...        ...       ...        ...   
2875  0.50         10           300     5        0.5  0.883333   1.000000   
2876  0.50         10           300     5        0.6  0.883333   1.000000   
2877  0.50         10           300     5        0.7  0.883333   1.000000   
2878  0.50         10           300     5        0.8  0.866667   1.000000  

C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:42:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\haha9\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:42:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


KNN

In [15]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.random_projection import SparseRandomProjection
from sklearn.neighbors import KNeighborsClassifier
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Manual Grid Search KNN with Cross-Validation

def flastThreshold(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
        # Initialize KNeighborsClassifier with current hyperparameter combination
        knn_model = KNeighborsClassifier(
            n_neighbors=param_dict['n_neighbors'],
            weights=param_dict['weights'],
            algorithm=param_dict['algorithm'],
            leaf_size=param_dict['leaf_size'],
            p=param_dict['p'],
            n_jobs=-1
        )
        
        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the model
            knn_model.fit(X_train, y_train)

            # Predict probabilities on test set
            if hasattr(knn_model, "predict_proba"):
                y_pred_proba = knn_model.predict_proba(X_test)
            else:
                # If predict_proba is not available, use distance-based probabilities
                distances, indices = knn_model.kneighbors(X_test)
                weights = knn_model._get_weights(distances)
                y_pred_proba = np.zeros((X_test.shape[0], 2))
                for i, neighbors in enumerate(indices):
                    neighbor_labels = y_train[neighbors]
                    if weights is None:
                        proba = np.bincount(neighbor_labels, minlength=2) / knn_model.n_neighbors
                    else:
                        proba = np.bincount(neighbor_labels, weights=weights[i], minlength=2) / weights[i].sum()
                    y_pred_proba[i] = proba

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred,  zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=1)
                recall = recall_score(y_test, y_pred,  zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'n_neighbors': param_dict['n_neighbors'],
                    'weights': param_dict['weights'],
                    'algorithm': param_dict['algorithm'],
                    'leaf_size': param_dict['leaf_size'],
                    'p': param_dict['p'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return param_dict, None

    # Save the results for each combination, threshold, and fold
    df_results = pd.DataFrame(metrics_per_combination)
    outFile = f"{combination_label}-params-knn-{n_splits}-folds-Threshold-allKfold-wogridsearch.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"KNN analysis completed. Results saved to: {outFile}")
    return param_dict, df_results



if __name__ == "__main__":

    param_grid = {
        'n_neighbors': [3, 5, 7, 9,11,15,20],           # Number of neighbors to use
        'weights': ['uniform', 'distance'],    # Weight function used in prediction
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute the nearest neighbors
        'leaf_size': [30, 50],                 # Leaf size passed to BallTree or KDTree
        'p': [1, 2],                           # Power parameter for the Minkowski metric
    }

    # Parameters setup for the first combination
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

    # Create separate result directories for equal and larger non-flaky combinations
    outDirEqual = "results/equal_flaky_nonflaky/"
    outDirLarger = "results/larger_nonflaky/"
    os.makedirs(outDirEqual, exist_ok=True)
    os.makedirs(outDirLarger, exist_ok=True)

    # Create separate extract directories for each combination to avoid file confusion
    extractDirEqual = "extracted/equal_flaky_nonflaky/"
    extractDirLarger = "extracted/larger_nonflaky/"
    os.makedirs(extractDirEqual, exist_ok=True)
    os.makedirs(extractDirLarger, exist_ok=True)

    # Perform KNN analysis for the first combination (flaky vs smaller non-flaky)
    print("Starting KNN analysis for flaky vs smaller non-flaky files (47 each)...")
    best_params_5folds_1, df_results_5folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 5, dim=100, eps=0.3, combination_label="equal", param_grid=param_grid)
    
    print("Best results for 5-fold on equal combination:")
    print(df_results_5folds_1)

    # Perform KNN analysis for the second combination (flaky vs larger non-flaky)
    print("Starting KNN analysis for flaky vs larger non-flaky files...")
    best_params_5folds_2, df_results_5folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 5, dim=100, eps=0.3, combination_label="larger", param_grid=param_grid)
    
    print("Best results for 5-fold on larger combination:")
    print(df_results_5folds_2)



Starting KNN analysis for flaky vs smaller non-flaky files (47 each)...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto', 'leaf_size': 30, 'p': 1}
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto', 'leaf_size': 30, 'p': 2}
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto', 'leaf_size': 50, 'p': 1}
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto', 'leaf_size': 50, 'p': 2}
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'ball_tree', 'leaf_size': 30, 'p': 1}
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'ball_tree', 'leaf_size': 30, 'p': 2}


C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warni

Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'ball_tree', 'leaf_size': 50, 'p': 1}
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'ball_tree', 'leaf_size': 50, 'p': 2}


C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warni

Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'kd_tree', 'leaf_size': 30, 'p': 1}
Training with parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'kd_tree', 'leaf_size': 30, 'p': 2}


...

Training with parameters: {'n_neighbors': 20, 'weights': 'distance', 'algorithm': 'kd_tree', 'leaf_size': 30, 'p': 2}
Training with parameters: {'n_neighbors': 20, 'weights': 'distance', 'algorithm': 'kd_tree', 'leaf_size': 50, 'p': 1}


C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warni

Training with parameters: {'n_neighbors': 20, 'weights': 'distance', 'algorithm': 'kd_tree', 'leaf_size': 50, 'p': 2}
Training with parameters: {'n_neighbors': 20, 'weights': 'distance', 'algorithm': 'brute', 'leaf_size': 30, 'p': 1}


C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\neighbors\_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


Training with parameters: {'n_neighbors': 20, 'weights': 'distance', 'algorithm': 'brute', 'leaf_size': 30, 'p': 2}
Training with parameters: {'n_neighbors': 20, 'weights': 'distance', 'algorithm': 'brute', 'leaf_size': 50, 'p': 1}
Training with parameters: {'n_neighbors': 20, 'weights': 'distance', 'algorithm': 'brute', 'leaf_size': 50, 'p': 2}
KNN analysis completed. Results saved to: larger-params-knn-5-folds-Threshold-allKfold-wogridsearch.csv
Best results for 5-fold on larger combination:
       n_neighbors   weights algorithm  leaf_size  p  fold  threshold  \
0                3   uniform      auto         30  1     1        0.1   
1                3   uniform      auto         30  1     1        0.2   
2                3   uniform      auto         30  1     1        0.3   
3                3   uniform      auto         30  1     1        0.4   
4                3   uniform      auto         30  1     1        0.5   
...            ...       ...       ...        ... ..   ...     

SVM

In [16]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.random_projection import SparseRandomProjection
from sklearn.svm import SVC
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Manual Grid Search SVM with Cross-Validation

def flastThreshold(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
        # Initialize SVM with current hyperparameter combination
        svm_model = SVC(
            C=param_dict['C'],
            kernel=param_dict['kernel'],
            probability=True,  # Enable probability estimates
            random_state=42
        )
        
        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the model
            try:
                svm_model.fit(X_train, y_train)
            except Exception as e:
                print(f"Failed to train SVM with parameters {param_dict} on fold {fold+1}: {e}")
                continue

            # Predict probabilities on test set
            try:
                y_pred_proba = svm_model.predict_proba(X_test)
            except Exception as e:
                print(f"Failed to predict probabilities with SVM on fold {fold+1}: {e}")
                continue

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred,  zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred,  zero_division=1)
                recall = recall_score(y_test, y_pred,  zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'C': param_dict['C'],
                    'kernel': param_dict['kernel'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return param_dict, None

    # Save the results for each combination, threshold, and fold
    df_results = pd.DataFrame(metrics_per_combination)
    outFile = f"{combination_label}-params-svm-{n_splits}-folds-Threshold-allKfold-wogridsearch.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"SVM analysis completed. Results saved to: {outFile}")
    return param_dict, df_results



if __name__ == "__main__":

    param_grid = {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization parameter
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid']  # Kernel types
    }

    # Parameters setup for the first combination
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

    # Create separate result directories for equal and larger non-flaky combinations
    outDirEqual = "results/equal_flaky_nonflaky/"
    outDirLarger = "results/larger_nonflaky/"
    os.makedirs(outDirEqual, exist_ok=True)
    os.makedirs(outDirLarger, exist_ok=True)

    # Create separate extract directories for each combination to avoid file confusion
    extractDirEqual = "extracted/equal_flaky_nonflaky/"
    extractDirLarger = "extracted/larger_nonflaky/"
    os.makedirs(extractDirEqual, exist_ok=True)
    os.makedirs(extractDirLarger, exist_ok=True)

    # Perform SVM analysis for the first combination (flaky vs smaller non-flaky)
    print("Starting SVM analysis for flaky vs smaller non-flaky files (47 each)...")
    best_params_5folds_1, df_results_5folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 5, dim=100, eps=0.3, combination_label="equal", param_grid=param_grid)
    
    print("Best results for 5-fold on equal combination:")
    print(df_results_5folds_1)

    # Perform SVM analysis for the second combination (flaky vs larger non-flaky)
    print("Starting SVM analysis for flaky vs larger non-flaky files...")
    best_params_5folds_2, df_results_5folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 5, dim=100, eps=0.3, combination_label="larger", param_grid=param_grid)
    
    print("Best results for 5-fold on larger combination:")
    print(df_results_5folds_2)

   


Starting SVM analysis for flaky vs smaller non-flaky files (47 each)...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Training with parameters: {'C': 0.01, 'kernel': 'linear'}
Training with parameters: {'C': 0.01, 'kernel': 'rbf'}
Training with parameters: {'C': 0.01, 'kernel': 'poly'}
Training with parameters: {'C': 0.01, 'kernel': 'sigmoid'}
Training with parameters: {'C': 0.1, 'kernel': 'linear'}
Training with parameters: {'C': 0.1, 'kernel': 'rbf'}
Training with parameters: {'C': 0.1, 'kernel': 'poly'}
Training with parameters: {'C': 0.1, 'kernel': 'sigmoid'}
Training with parameters: {'C': 1.0, 'kernel': 'linear'}
Training with parameters: {'C': 1.0, 'kernel': 'rbf'}
Training with parameters: {'C': 1.0, 'kernel': 'poly'}
Training with parameters: {'C': 1.0, 'kernel': 'sigmoid'}
Training with parameters: {'C': 10.0, 'kernel': 'linear'}
Training with parameters: {'C': 10.0, 'kernel': 'rbf'}
Training with parameters: {'C': 10.0, 'kernel

NB

In [6]:
#unlance without adjustment
#

In [17]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold
from sklearn.random_projection import SparseRandomProjection
from sklearn.naive_bayes import MultinomialNB
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Manual Grid Search Naive Bayes with Cross-Validation

def flastThreshold(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization without dimensionality reduction
    Z = flastVectorization(dataPoints, dim=0, eps=0)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")
        
        # Initialize MultinomialNB with current hyperparameter combination
        nb_model = MultinomialNB(
            alpha=param_dict['alpha']
        )
        
        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Train the model
            nb_model.fit(X_train, y_train)

            # Predict probabilities on test set
            y_pred_proba = nb_model.predict_proba(X_test)

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred, zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=1)
                recall = recall_score(y_test, y_pred, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'alpha': param_dict['alpha'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return param_dict, None

    # Save the results for each combination, threshold, and fold
    df_results = pd.DataFrame(metrics_per_combination)
    outFile = f"{combination_label}-params-nb-{n_splits}-folds-Threshold-allKfold-wogridsearch.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Naive Bayes analysis completed. Results saved to: {outFile}")
    return param_dict, df_results



if __name__ == "__main__":

    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0],  # Additive smoothing parameter
    }

    # Parameters setup for the first combination
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

    # Create separate result directories for equal and larger non-flaky combinations
    outDirEqual = "results/equal_flaky_nonflaky/"
    outDirLarger = "results/larger_nonflaky/"
    os.makedirs(outDirEqual, exist_ok=True)
    os.makedirs(outDirLarger, exist_ok=True)

    # Create separate extract directories for each combination to avoid file confusion
    extractDirEqual = "extracted/equal_flaky_nonflaky/"
    extractDirLarger = "extracted/larger_nonflaky/"
    os.makedirs(extractDirEqual, exist_ok=True)
    os.makedirs(extractDirLarger, exist_ok=True)

    # Perform Naive Bayes analysis for the first combination (flaky vs smaller non-flaky)
    print("Starting Naive Bayes analysis for flaky vs smaller non-flaky files (47 each)...")
    best_params_5folds_1, df_results_5folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 5, dim=100, eps=0.3, combination_label="equal", param_grid=param_grid)
    
    print("Best results for 5-fold on equal combination:")
    print(df_results_5folds_1)

    # Perform Naive Bayes analysis for the second combination (flaky vs larger non-flaky)
    print("Starting Naive Bayes analysis for flaky vs larger non-flaky files...")
    best_params_5folds_2, df_results_5folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 5, dim=100, eps=0.3, combination_label="larger", param_grid=param_grid)
    
    print("Best results for 5-fold on larger combination:")
    print(df_results_5folds_2)



Starting Naive Bayes analysis for flaky vs smaller non-flaky files (47 each)...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Training with parameters: {'alpha': 0.001}
Training with parameters: {'alpha': 0.01}
Training with parameters: {'alpha': 0.1}
Training with parameters: {'alpha': 1.0}
Training with parameters: {'alpha': 10.0}
Naive Bayes analysis completed. Results saved to: equal-params-nb-5-folds-Threshold-allKfold-wogridsearch.csv
Best results for 5-fold on equal combination:
      alpha  fold  threshold  accuracy  precision    recall        f1  \
0     0.001     1        0.1  0.736842   0.857143  0.600000  0.705882   
1     0.001     1        0.2  0.736842   0.857143  0.600000  0.705882   
2     0.001     1        0.3  0.736842   0.857143  0.600000  0.705882   
3     0.001     1        0.4  0.736842   0.857143  0.600000  0.705882   
4     0.001     1        0.5  0.736842   0.857143  0.600000  0.705882   
..      ...   ...      

In [18]:
import os
import pandas as pd

# Function to extract the best results from the CSV files of each model
def extract_best_results(model_name, combination, fold_label, csv_file):
    """
    Extracts the best result from the CSV file for a model.

    Parameters:
    - model_name: The name of the model (e.g., "KNN", "SVM")
    - combination: The combination of flaky and non-flaky files (e.g., "equal", "larger")
    - fold_label: Number of folds (e.g., "5-fold" or "3-fold")
    - csv_file: The path to the CSV file containing the model's results

    Returns:
    A dictionary containing the best results for the model, combination, and fold.
    """
    if not os.path.exists(csv_file):
        print(f"CSV file for {model_name} ({combination}, {fold_label}) does not exist: {csv_file}")
        return None

    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    if df.empty:
        print(f"CSV file for {model_name} ({combination}, {fold_label}) is empty: {csv_file}")
        return None
    
    # Identify the metric columns
    metric_columns = ['accuracy', 'precision', 'recall', 'f1', 'mcc']
    # Identify parameter columns (exclude known metric columns and 'fold' and 'threshold')
    parameter_columns = [col for col in df.columns if col not in metric_columns + ['fold', 'threshold']]
    
    # Group by hyperparameters, fold, and threshold, then find the row with the highest F1 score
    idx = df.groupby(parameter_columns + ['fold', 'threshold'])['f1'].idxmax()
    df_best = df.loc[idx]
    
    # Now, find the overall best result (highest F1 score)
    best_row = df_best.loc[df_best['f1'].idxmax()]
    
    # Extract metrics
    accuracy = best_row['accuracy']
    precision = best_row['precision']
    recall = best_row['recall']
    f1 = best_row['f1']
    mcc = best_row.get('mcc', None)  # Get MCC if available
    
    # Extract parameters
    parameters = {col: best_row[col] for col in parameter_columns}
    parameters['threshold'] = best_row['threshold']
    parameters['fold'] = int(best_row['fold'])
    
    # Create a combined model name
    combined_model_name = f"{combination} {model_name}"
    
    # Collect the best results into a dictionary
    best_results = {
        'Model': combined_model_name,
        'Fold': fold_label,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'MCC': mcc,
        'Parameters': parameters
    }
    
    return best_results

# Function to gather and print/save the best results from a single combination
def gather_best_results_for_combination(models_results_dir, output_file_prefix, combination):
    """
    Gathers the best results from all models for a specific combination (e.g., "equal", "larger") and writes them to separate CSV files.

    Parameters:
    - models_results_dir: Directory where the model result CSV files are stored for the combination.
    - output_file_prefix: Prefix for the output CSV files to store the best results for both 'wogridsearch' and 'one_hyperparameter'.
    - combination: The combination name (e.g., "equal", "larger").
    """
    # List of models and their corresponding result file patterns
    models = {
        'KNN': 'params-knn',
        'SVM': 'params-svm',
        'Naive Bayes': 'params-nb',
        'XGBoost': 'params-xgb',
        'Random Forest': 'params-rf',
        'Decision Tree': 'params-dt'
    }

    # Initialize an empty list to store the best results from each model and fold for both file types
    best_results_wogridsearch = []
    best_results_one_hyperparameter = []

    # Iterate over each model and its result files for both 5-fold and 3-fold
    for model_name, base_filename in models.items():
        for n_splits in [5, 3]:
            fold_label = f"{n_splits}-fold"
            
            # Construct the CSV filenames for both "wogridsearch" and "one_hyperparameter"
            csv_file_wogridsearch = f"{combination}-{base_filename}-{n_splits}-folds-Threshold-allKfold-wogridsearch.csv"
            csv_file_one_hyperparameter = f"{combination}-{base_filename}-{n_splits}-folds-Threshold-allKfold_one_hyperparameter.csv"
            
            # Get full paths
            full_csv_path_wogridsearch = os.path.join(models_results_dir, csv_file_wogridsearch)
            full_csv_path_one_hyperparameter = os.path.join(models_results_dir, csv_file_one_hyperparameter)

            # Extract the best results from both files
            best_result_wogridsearch = extract_best_results(model_name, combination, fold_label, full_csv_path_wogridsearch)
            best_result_one_hyperparameter = extract_best_results(model_name, combination, fold_label, full_csv_path_one_hyperparameter)

            # Append results to respective lists
            if best_result_wogridsearch:
                best_results_wogridsearch.append(best_result_wogridsearch)
            if best_result_one_hyperparameter:
                best_results_one_hyperparameter.append(best_result_one_hyperparameter)

    # Save the best results to CSV files
    if best_results_wogridsearch:
        best_results_df_wogridsearch = pd.DataFrame(best_results_wogridsearch)
        output_file_wogridsearch = f"{output_file_prefix}_wogridsearch.csv"
        best_results_df_wogridsearch.to_csv(output_file_wogridsearch, index=False)
        print(f"Best results for {combination} combination (wogridsearch) saved to: {output_file_wogridsearch}")

    if best_results_one_hyperparameter:
        best_results_df_one_hyperparameter = pd.DataFrame(best_results_one_hyperparameter)
        output_file_one_hyperparameter = f"{output_file_prefix}_one_hyperparameter.csv"
        best_results_df_one_hyperparameter.to_csv(output_file_one_hyperparameter, index=False)
        print(f"Best results for {combination} combination (one_hyperparameter) saved to: {output_file_one_hyperparameter}")
    
    # Print the best results as a table for both cases
    if best_results_wogridsearch:
        print(f"\nBest Results from All Models for {combination} Combination (wogridsearch):")
        print(best_results_df_wogridsearch.to_string(index=False))

    if best_results_one_hyperparameter:
        print(f"\nBest Results from All Models for {combination} Combination (one_hyperparameter):")
        print(best_results_df_one_hyperparameter.to_string(index=False))

# Example usage
if __name__ == "__main__":
    # Directories where the model result CSV files are stored for each combination
    equal_results_dir = 'results/equal_flaky_nonflaky/'
    larger_results_dir = 'results/larger_nonflaky/'

    # Prefixes for the output CSV files
    equal_output_prefix = "best_results_equal_combination"
    larger_output_prefix = "best_results_larger_combination"

    # Gather and save the best results for the equal combination
    gather_best_results_for_combination(equal_results_dir, equal_output_prefix, "equal")

    # Gather and save the best results for the larger combination
    gather_best_results_for_combination(larger_results_dir, larger_output_prefix, "larger")



CSV file for KNN (equal, 3-fold) does not exist: results/equal_flaky_nonflaky/equal-params-knn-3-folds-Threshold-allKfold-wogridsearch.csv
CSV file for SVM (equal, 3-fold) does not exist: results/equal_flaky_nonflaky/equal-params-svm-3-folds-Threshold-allKfold-wogridsearch.csv
CSV file for Naive Bayes (equal, 5-fold) does not exist: results/equal_flaky_nonflaky/equal-params-nb-5-folds-Threshold-allKfold_one_hyperparameter.csv
CSV file for Naive Bayes (equal, 3-fold) does not exist: results/equal_flaky_nonflaky/equal-params-nb-3-folds-Threshold-allKfold-wogridsearch.csv
CSV file for Naive Bayes (equal, 3-fold) does not exist: results/equal_flaky_nonflaky/equal-params-nb-3-folds-Threshold-allKfold_one_hyperparameter.csv
CSV file for XGBoost (equal, 3-fold) does not exist: results/equal_flaky_nonflaky/equal-params-xgb-3-folds-Threshold-allKfold-wogridsearch.csv
CSV file for Random Forest (equal, 3-fold) does not exist: results/equal_flaky_nonflaky/equal-params-rf-3-folds-Threshold-allKfol

PermissionError: [Errno 13] Permission denied: 'best_results_larger_combination_wogridsearch.csv'